# Analysis on Lyft's Bay Wheels Bike Sharing Program

# Table of Contents
- Gathering
- Assessing
- Cleaning

## Gathering

In [1]:
import os
import requests
import zipfile
import io
import glob
import numpy as np
import pandas as pd

In [2]:
#Create folder to store files
folder_name = 'raw_files'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [3]:
#Create function to download and extract files from source
def extract(x):
    #download file and extract
    response= requests.get(x)
    try:
        zp = zipfile.ZipFile(io.BytesIO(response.content))
        zp.extractall(folder_name)
    
    #if not a valid link, ignore and continue the script
    except zipfile.BadZipFile:
        pass

In [4]:
#Get the month year component of the URL
month_year = []
for i in range(2017, 2020):
    if i == 2017:
        month_year.append(str(i))
    else:
        for j in range(1,13):
            month_year.append(str(i) + str(j).zfill(2))  #make sure MM is in double digit

In [ ]:
#Re-create URL and download link
for i in month_year:
    if i < '201905':
        old_url = 'https://s3.amazonaws.com/baywheels-data/{}-fordgobike-tripdata.csv.zip'.format(i)
        extract(old_url)
    else:
        new_url = 'https://s3.amazonaws.com/baywheels-data/{}-baywheels-tripdata.csv.zip'.format(i)
        extract(new_url)

In [5]:
#Concatenate all csv into one dataframe
master = pd.concat((pd.read_csv(files, dtype={'bike_share_for_all_trip': object, 'rental_access_method': object}) for files in glob.glob(folder_name+'/*.csv')), sort=False, ignore_index=True)

In [6]:
#Clone master file
df = master.copy()

In [7]:
#Ensure dataframe is working
df.sample(10)

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,bike_share_for_all_trip,rental_access_method
3100442,785,2019-04-17 17:09:31.1570,2019-04-17 17:22:37.0430,90.0,Townsend St at 7th St,37.771058,-122.402717,15.0,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,547,Subscriber,No,NaN
1439101,621,2018-07-17 07:51:31.5210,2018-07-17 08:01:52.6220,27.0,Beale St at Harrison St,37.788059,-122.391865,104.0,4th St at 16th St,37.767045,-122.390833,688,Subscriber,No,NaN
1251573,265,2018-06-14 15:39:11.4810,2018-06-14 15:43:36.6510,182.0,19th Street BART Station,37.809013,-122.268247,195.0,Bay Pl at Vernon St,37.812314,-122.260779,57,Subscriber,No,NaN
2454711,565,2019-01-23 13:12:35.8470,2019-01-23 13:22:01.2190,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,47.0,4th St at Harrison St,37.780955,-122.399749,4432,Subscriber,No,NaN
910334,470,2018-04-13 19:19:44.0580,2018-04-13 19:27:34.0610,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,67.0,San Francisco Caltrain Station 2 (Townsend St...,37.776639,-122.395526,423,Subscriber,No,NaN
453716,573,2017-08-09 08:54:34.1480,2017-08-09 09:04:07.8600,22.0,Howard St at Beale St,37.789756,-122.394643,6.0,The Embarcadero at Sansome St,37.804770,-122.403234,2683,Subscriber,NaN,NaN
1193322,292,2018-06-23 15:28:41.5230,2018-06-23 15:33:33.9090,5.0,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,64.0,5th St at Brannan St,37.776754,-122.399018,3809,Subscriber,Yes,NaN
4530927,230,2019-10-03 17:11:19.8470,2019-10-03 17:15:10.6470,79.0,7th St at Brannan St,37.773492,-122.403673,30.0,San Francisco Caltrain (Townsend St at 4th St),37.776598,-122.395282,2139,Subscriber,No,NaN
4088652,1035,2019-08-02 08:07:32.8980,2019-08-02 08:24:48.7000,345.0,Hubbell St at 16th St,37.766483,-122.398279,16.0,Steuart St at Market St,37.794130,-122.394430,1511,Subscriber,No,NaN
4600921,1190,2019-11-20 14:48:49.3120,2019-11-20 15:08:39.3340,258.0,University Ave at Oxford St,37.872355,-122.266447,210.0,45th St at Manila,37.833294,-122.256224,11717,Customer,No,NaN


## Assessing

In [8]:
df.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4740302 entries, 0 to 4740301
Data columns (total 15 columns):
duration_sec               4740302 non-null int64
start_time                 4740302 non-null object
end_time                   4740302 non-null object
start_station_id           4668816 non-null float64
start_station_name         4669413 non-null object
start_station_latitude     4740302 non-null float64
start_station_longitude    4740302 non-null float64
end_station_id             4666976 non-null float64
end_station_name           4667526 non-null object
end_station_latitude       4740302 non-null float64
end_station_longitude      4740302 non-null float64
bike_id                    4740302 non-null int64
user_type                  4740302 non-null object
bike_share_for_all_trip    4127445 non-null object
rental_access_method       93157 non-null object
dtypes: float64(6), int64(2), object(7)
memory usage: 542.5+ MB


In [9]:
df.user_type.unique(), df.bike_share_for_all_trip.unique(), df.rental_access_method.unique()

(array(['Customer', 'Subscriber'], dtype=object),
 array([nan, 'No', 'Yes'], dtype=object),
 array([nan, 'app', 'clipper'], dtype=object))

In [10]:
df.bike_id.value_counts().describe()

count    12109.000000
mean       391.469320
std        436.445047
min          1.000000
25%         74.000000
50%        193.000000
75%        610.000000
max       2000.000000
Name: bike_id, dtype: float64

In [11]:
df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,bike_share_for_all_trip,rental_access_method
0,80110,2017-12-31 16:57:39.6540,2018-01-01 15:12:50.2450,74.0,Laguna St at Hayes St,37.776435,-122.426244,43.0,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer,NaN,NaN
1,78800,2017-12-31 15:56:34.8420,2018-01-01 13:49:55.6170,284.0,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96.0,Dolores St at 15th St,37.766210,-122.426614,88,Customer,NaN,NaN
2,45768,2017-12-31 22:45:48.4110,2018-01-01 11:28:36.8830,245.0,Downtown Berkeley BART,37.870348,-122.267764,245.0,Downtown Berkeley BART,37.870348,-122.267764,1094,Customer,NaN,NaN
3,62172,2017-12-31 17:31:10.6360,2018-01-01 10:47:23.5310,60.0,8th St at Ringold St,37.774520,-122.409449,5.0,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2831,Customer,NaN,NaN
4,43603,2017-12-31 14:23:14.0010,2018-01-01 02:29:57.5710,239.0,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247.0,Fulton St at Bancroft Way,37.867789,-122.265896,3167,Subscriber,NaN,NaN


In [12]:
#Check for missing coordinates
df.query('start_station_latitude == 0 | end_station_latitude ==0').head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,bike_share_for_all_trip,rental_access_method
3016391,2365,2019-04-30 18:21:42.6100,2019-04-30 19:01:08.1460,50.0,2nd St at Townsend St,37.780526,-122.390288,420.0,SF Test Station,0.0,0.0,1612,Subscriber,No,NaN
3017157,1964,2019-04-30 17:31:19.6460,2019-04-30 18:04:04.5180,98.0,Valencia St at 16th St,37.765052,-122.421866,420.0,SF Test Station,0.0,0.0,3507,Subscriber,No,NaN
3017311,546,2019-04-30 17:45:38.4520,2019-04-30 17:54:45.2580,370.0,Jones St at Post St,37.787327,-122.413278,420.0,SF Test Station,0.0,0.0,1142,Subscriber,Yes,NaN
3017390,630,2019-04-30 17:39:33.5070,2019-04-30 17:50:04.3140,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,420.0,SF Test Station,0.0,0.0,3500,Subscriber,No,NaN
3017843,433,2019-04-30 17:17:10.4860,2019-04-30 17:24:23.9190,11.0,Davis St at Jackson St,37.797280,-122.398436,420.0,SF Test Station,0.0,0.0,1181,Subscriber,No,NaN


In [13]:
#Check for missing station id
df[df.end_station_id.isnull()|df.start_station_id.isnull()].head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,bike_share_for_all_trip,rental_access_method
1143064,5572,2018-06-30 20:58:31.8550,2018-06-30 22:31:24.5930,NaN,NaN,37.40,-121.94,NaN,NaN,37.40,-121.94,4202,Customer,No,NaN
1143076,573,2018-06-30 22:12:27.5110,2018-06-30 22:22:01.1740,NaN,NaN,37.40,-121.94,NaN,NaN,37.40,-121.93,4095,Subscriber,Yes,NaN
1143093,850,2018-06-30 22:01:25.5850,2018-06-30 22:15:36.1510,NaN,NaN,37.41,-121.94,NaN,NaN,37.41,-121.95,4122,Customer,No,NaN
1143129,562,2018-06-30 21:52:05.1180,2018-06-30 22:01:27.4400,NaN,NaN,37.41,-121.94,NaN,NaN,37.42,-121.94,4184,Subscriber,No,NaN
1143156,819,2018-06-30 21:37:34.5060,2018-06-30 21:51:13.7290,NaN,NaN,37.41,-121.94,NaN,NaN,37.41,-121.96,4137,Customer,No,NaN


### Action Plan

#### Quality Related
- Convert *start_time* and *end_time* to datefime
- Convert *start_station_id*, *end_station_id* and *bike_id* to strings
- Remove entries with missing station id

#### Tidiness Related
- Create new duration column in minutes
- Create new datetime columns
- Calculate distance based off coordinates (latitude and longitude)
- Create bins based on time of usage (Morning, Afternoon, Evening, Night)

## Cleaning

### Convert *start_time* and *end_time* to datefime

In [14]:
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

### Convert *start_station_id*, *end_station_id* and *bike_id* to strings

In [15]:
df['start_station_id'] = df['start_station_id'].astype(str).str.replace('\.0','')
df['end_station_id'] = df['end_station_id'].astype(str).str.replace('\.0','')
df['bike_id'] = df['bike_id'].astype(str)

**Test**

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4740302 entries, 0 to 4740301
Data columns (total 15 columns):
duration_sec               int64
start_time                 datetime64[ns]
end_time                   datetime64[ns]
start_station_id           object
start_station_name         object
start_station_latitude     float64
start_station_longitude    float64
end_station_id             object
end_station_name           object
end_station_latitude       float64
end_station_longitude      float64
bike_id                    object
user_type                  object
bike_share_for_all_trip    object
rental_access_method       object
dtypes: datetime64[ns](2), float64(4), int64(1), object(8)
memory usage: 542.5+ MB


### Remove entries with missing station id

In [17]:
df.dropna(subset=['start_station_id', 'end_station_id'], inplace=True)

**Test**

In [18]:
df[df.end_station_id.isnull()|df.start_station_id.isnull()].head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,bike_share_for_all_trip,rental_access_method


### Create new duration column in minutes

In [19]:
df['duration_min'] = (df.duration_sec/60).round(2)

### Create new datetime columns

In [20]:
df['start_year'] = df.start_time.dt.year
df['start_month'] = df.start_time.dt.month
df['start_week'] = df.start_time.dt.weekday_name
df['start_date'] = df.start_time.dt.date
df['start_hour'] = df.start_time.dt.hour

### Calculate distance based off coordinates

In [21]:
#Function for the Haversine Formula
#All credit goes to @derricw: https://stackoverflow.com/a/29546836

def haversine(lon1, lat1, lon2, lat2):
    
    """    
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367.8 * c # Units in km.  3959.87433 for miles
    return km

In [22]:
#Create column
df['distance'] = haversine(df['start_station_longitude'],df['start_station_latitude'],df['end_station_longitude'],df['end_station_latitude'])

In [23]:
#Could see that there are outliners due to missing coordinates
df.query('start_station_latitude == 0 | end_station_latitude ==0').head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,...,user_type,bike_share_for_all_trip,rental_access_method,duration_min,start_year,start_month,start_week,start_date,start_hour,distance
3016391,2365,2019-04-30 18:21:42.610,2019-04-30 19:01:08.146,50,2nd St at Townsend St,37.780526,-122.390288,420,SF Test Station,0.0,...,Subscriber,No,NaN,39.42,2019,4,Tuesday,2019-04-30,18,12786.379906
3017157,1964,2019-04-30 17:31:19.646,2019-04-30 18:04:04.518,98,Valencia St at 16th St,37.765052,-122.421866,420,SF Test Station,0.0,...,Subscriber,No,NaN,32.73,2019,4,Tuesday,2019-04-30,17,12789.588682
3017311,546,2019-04-30 17:45:38.452,2019-04-30 17:54:45.258,370,Jones St at Post St,37.787327,-122.413278,420,SF Test Station,0.0,...,Subscriber,Yes,NaN,9.10,2019,4,Tuesday,2019-04-30,17,12787.988074
3017390,630,2019-04-30 17:39:33.507,2019-04-30 17:50:04.314,21,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,420,SF Test Station,0.0,...,Subscriber,No,NaN,10.50,2019,4,Tuesday,2019-04-30,17,12786.874940
3017843,433,2019-04-30 17:17:10.486,2019-04-30 17:24:23.919,11,Davis St at Jackson St,37.797280,-122.398436,420,SF Test Station,0.0,...,Subscriber,No,NaN,7.22,2019,4,Tuesday,2019-04-30,17,12786.372190


In [24]:
df = df.query('start_station_latitude != 0 & end_station_latitude !=0')
df.reset_index(drop=True).head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,...,user_type,bike_share_for_all_trip,rental_access_method,duration_min,start_year,start_month,start_week,start_date,start_hour,distance
0,80110,2017-12-31 16:57:39.654,2018-01-01 15:12:50.245,74,Laguna St at Hayes St,37.776435,-122.426244,43,San Francisco Public Library (Grove St at Hyde...,37.778768,...,Customer,NaN,NaN,1335.17,2017,12,Sunday,2017-12-31,16,0.942458
1,78800,2017-12-31 15:56:34.842,2018-01-01 13:49:55.617,284,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96,Dolores St at 15th St,37.766210,...,Customer,NaN,NaN,1313.33,2017,12,Sunday,2017-12-31,15,3.068185
2,45768,2017-12-31 22:45:48.411,2018-01-01 11:28:36.883,245,Downtown Berkeley BART,37.870348,-122.267764,245,Downtown Berkeley BART,37.870348,...,Customer,NaN,NaN,762.80,2017,12,Sunday,2017-12-31,22,0.000000
3,62172,2017-12-31 17:31:10.636,2018-01-01 10:47:23.531,60,8th St at Ringold St,37.774520,-122.409449,5,Powell St BART Station (Market St at 5th St),37.783899,...,Customer,NaN,NaN,1036.20,2017,12,Sunday,2017-12-31,17,1.046096
4,43603,2017-12-31 14:23:14.001,2018-01-01 02:29:57.571,239,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247,Fulton St at Bancroft Way,37.867789,...,Subscriber,NaN,NaN,726.72,2017,12,Sunday,2017-12-31,14,0.636020


**Test**

In [25]:
df.query('start_station_latitude == 0 | end_station_latitude ==0')

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,...,user_type,bike_share_for_all_trip,rental_access_method,duration_min,start_year,start_month,start_week,start_date,start_hour,distance


### Create bins based on time of usage

In [26]:
#Definition Morning: 5:00 - 11:59, Afternoon: 12:00 to 16:59, Evening: 17:00 - 20:59, Night: 21- 4:59

bin_name = ['NightX','Morning', 'Afternoon','Evening','Night']
bin_edges= [0,5,12,17,21,24]
df['period'] = pd.cut(df['start_hour'], bin_edges, labels=bin_name, right=False).str.replace('NightX','Night')